# Functioneer Advanced Examples

This notebook provides advanced examples that combine many features of the `functioneer` library.
Currently this notebook is the primary form of documentation. By the end you will have witnessed the computational power of this fully armed and fully operational library.

## Table of Contents
1. [Making a surface plot](#example-1)
2. [Optimization with post-processing](#example-2)


In [1]:
# Setup: Import libraries and define example functions
import functioneer as fn
import numpy as np
import plotly.graph_objects as go
import time

# Rosenbrock function (known minimum: value=0 @ x=1, y=1, a=1, b=100)
def rosenbrock(x, y, a, b):
    return (a - x)**2 + b * (y - x**2)**2

### <span id='example-1'>Example 1: Surface Plot</span>

**Goal**: Plot the `rosenbrock` function over the `x` and `y` domain.

Note: Plotly wants the results formatted as a grid


In [4]:
analysis = fn.AnalysisModule({'a': 1, 'b': 100}) # Create new analysis
x_values = np.linspace(-2, 2, 20)
y_values = np.linspace(-1, 3, 20)
analysis.add.fork('x', x_values.tolist()) # Fork analysis, create branches for x=0, x=1, x=2
analysis.add.fork('y', y_values.tolist())
analysis.add.evaluate(func=rosenbrock) #
results = analysis.run()
print('Example 1 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock']])

X, Y = np.meshgrid(x_values, y_values)
Z = (1 - X)**2 + 100 * (Y - X**2)**2  # Rosenbrock for surface

# Create 3D surface plot
fig = go.Figure(data=[
    go.Surface(x=X, y=Y, z=Z, colorscale='Viridis', showscale=True),
    # go.Surface(
    #     x=results['df']['x'], 
    #     y=results['df']['y'],
    #     z=results['df']['rosenbrock'], 
    #     colorscale='Viridis', 
    #     showscale=True
    # )
    go.Scatter3d(
		x=results['df']['x'],
        y=results['df']['y'],
        z=results['df']['rosenbrock'],
        mode='lines+markers',
        marker=dict(size=4, color='red', symbol='circle')
	)
])

fig.update_layout(
    scene=dict(

        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='Rosenbrock Value'
	),
)

fig.show()


Example 1 Output:
     a    b    x         y   rosenbrock
0    1  100 -2.0 -1.000000  2509.000000
1    1  100 -2.0 -0.789474  2302.905817
2    1  100 -2.0 -0.578947  2105.675900
3    1  100 -2.0 -0.368421  1917.310249
4    1  100 -2.0 -0.157895  1737.808864
..  ..  ...  ...       ...          ...
395  1  100  2.0  2.157895   340.335180
396  1  100  2.0  2.368421   267.204986
397  1  100  2.0  2.578947   202.939058
398  1  100  2.0  2.789474   147.537396
399  1  100  2.0  3.000000   101.000000

[400 rows x 5 columns]


In [ ]:
def expensive_func(x, y):
    time.sleep(0.5)  # delay to simulate expensive func
    return x + y

analysis = fn.AnalysisModule({'x': 0, 'y': 0, 'a': 1, 'b': 100})
analysis.add.fork('a', (1, 2, 3, 4, 5, 6, 7, 8, 9))
analysis.add.fork('b', (0, 100, 200))
analysis.add.optimize(func=rosenbrock, opt_param_ids=('x', 'y'))
analysis.add.evaluate(func=expensive_func, condition=lambda y: y > 0.5)
results = analysis.run()
print('\nExample 7 Output:')
print(results['df'][['a', 'b', 'x', 'y', 'rosenbrock', 'expensive_func', 'runtime']])